In [1]:
# Load Libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import random
import numpy as np
from sklearn.metrics import confusion_matrix
import warnings
import matplotlib.pyplot as plt
import os
%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
D:\anaconda3\envs\py3-maxwell\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 100
num_classes = 10
batch_size = 32
learning_rate = 0.01
num_samples = 10000


# Set deterministic random seed
random_seed = 999
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # if use multi-GPU
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [3]:
current_path = os.getcwd()
print(current_path)

D:\2.CEM\maxwellfdfd-ai\mnist


In [4]:
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)

In [5]:
train_dataset = datasets.MNIST(root=f'./data/',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=False)
# random_idx = list(range(0, num_samples))
random_idx = np.random.permutation(num_samples)
train_subset = torch.utils.data.Subset(train_dataset, random_idx)

In [6]:
test_dataset = datasets.MNIST(root='data/',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_subset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [7]:
def lr_decay(step):
    epoch = step // (num_samples // batch_size)
    # print(f'step:{step}, epoch:{epoch}, num_samples:{num_samples}, batch size:{batch_size}')
    if epoch < 50:
        return 1.0
    else:
        return 0.1

In [8]:

# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(10*10*64, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [9]:

model = ConvNet(num_classes).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.2)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda = lr_decay, last_epoch=-1)
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()

        optimizer.step()
        scheduler.step()

        if (i+1) % 10 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Epoch [1/100], Step [10/313], Loss: 1.8950
Epoch [1/100], Step [20/313], Loss: 0.4045
Epoch [1/100], Step [30/313], Loss: 0.4124
Epoch [1/100], Step [40/313], Loss: 0.4208
Epoch [1/100], Step [50/313], Loss: 0.1912
Epoch [1/100], Step [60/313], Loss: 0.2436
Epoch [1/100], Step [70/313], Loss: 0.1473
Epoch [1/100], Step [80/313], Loss: 0.1865
Epoch [1/100], Step [90/313], Loss: 0.3506
Epoch [1/100], Step [100/313], Loss: 0.2825
Epoch [1/100], Step [110/313], Loss: 0.2203
Epoch [1/100], Step [120/313], Loss: 0.2203
Epoch [1/100], Step [130/313], Loss: 0.1057
Epoch [1/100], Step [140/313], Loss: 0.1035
Epoch [1/100], Step [150/313], Loss: 0.2091
Epoch [1/100], Step [160/313], Loss: 0.0954
Epoch [1/100], Step [170/313], Loss: 0.1677
Epoch [1/100], Step [180/313], Loss: 0.0652
Epoch [1/100], Step [190/313], Loss: 0.1691
Epoch [1/100], Step [200/313], Loss: 0.2603
Epoch [1/100], Step [210/313], Loss: 0.0753
Epoch [1/100], Step [220/313], Loss: 0.1475
Epoch [1/100], Step [230/313], Loss: 0.13